In [24]:
import os
import csv
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt


In [ ]:
# =================================================================
# 1. CONFIGURATION DES CHEMINS
# =================================================================

# Chemin racine du dataset
ROOT_TRAIN = "../../../27919209/MSLesSegDataset/train"

# Dossier racine pour les sorties
GLOBAL_OUTPUT_DIR = "Moments_Extraction_Results"
MODALITIES = ["T1", "T2", "FLAIR"]
ORDRE_CIBLE = 10

In [26]:
# =================================================================
# 2. MOTEUR MATHÉMATIQUE (STABLE & SAFE)
# =================================================================

def tchebichef_polynomials(N, order):
    M = np.zeros((order, N))
    x = np.arange(N)
    M[0, :] = 1.0
    if order > 1: M[1, :] = (2.0 * x - N + 1.0) / N
    for p in range(2, order):
        M[p, :] = ((2.0*p-1.0)*(2.0*x-N+1.0)/(p*N))*M[p-1,:] - ((p-1.0)/p)*(1.0-((p-1.0)**2/(N**2)))*M[p-2,:]
    return M

def calculate_rho(N, order):
    rho = np.zeros(order); rho[0] = N
    for p in range(1, order): rho[p] = (N**2-p**2)/(N**2) * (2*p-1)/(2*p+1) * rho[p-1]
    return rho

def decompose_2d(image, requested_order):
    H, W = image.shape
    actual_order_h, actual_order_w = min(requested_order, H), min(requested_order, W)
    M_h = tchebichef_polynomials(H, actual_order_h)
    M_w = tchebichef_polynomials(W, actual_order_w)
    rho_h, rho_w = calculate_rho(H, actual_order_h), calculate_rho(W, actual_order_w)
    T_raw = M_h @ image @ M_w.T
    C_pq = 1.0 / np.outer(rho_h, rho_w)
    T_pq_small = T_raw * C_pq
    T_final = np.zeros((requested_order, requested_order))
    T_final[:actual_order_h, :actual_order_w] = T_pq_small
    return np.round(T_final, 3) # Arrondi à 3 décimales dès le calcul

def reconstruct_2d(moments, shape_originale):
    H, W = shape_originale
    order = moments.shape[0]
    M_h = tchebichef_polynomials(H, order)
    M_w = tchebichef_polynomials(W, order)
    return M_h.T @ moments @ M_w

In [27]:
# =================================================================
# 3. BOUCLE MASTER (TOUS PATIENTS, TOUTES MODALITÉS)
# =================================================================

# Liste des patients (P1, P2, etc.)
patients = sorted([d for d in os.listdir(ROOT_TRAIN) if os.path.isdir(os.path.join(ROOT_TRAIN, d))])

for patient in patients:
    patient_path = os.path.join(ROOT_TRAIN, patient)
    # Liste des timepoints (T1, T2)
    timepoints = sorted([d for d in os.listdir(patient_path) if os.path.isdir(os.path.join(patient_path, d))])
    
    for tp in timepoints:
        tp_path = os.path.join(patient_path, tp)
        
        # Création du dossier de sortie spécifique pour ce patient et ce timepoint
        current_out_dir = os.path.join(GLOBAL_OUTPUT_DIR, patient, tp)
        os.makedirs(current_out_dir, exist_ok=True)
        
        # Chemin du masque (commun aux modalités d'un même timepoint)
        mask_fn = f"{patient}_{tp}_MASK.nii.gz"
        mask_full_path = os.path.join(tp_path, mask_fn)
        
        if not os.path.exists(mask_full_path):
            continue
            
        # Chargement et identification des lésions
        mask_sitk = sitk.ReadImage(mask_full_path)
        mask_bin = sitk.Cast(mask_sitk > 0, sitk.sitkUInt8)
        lesion_cc = sitk.RelabelComponent(sitk.ConnectedComponent(mask_bin)) # L1 = plus grosse
        mask_arr = sitk.GetArrayFromImage(lesion_cc)
        num_lesions = int(mask_arr.max())
        
        for modality in MODALITIES:
            img_fn = f"{patient}_{tp}_{modality}.nii.gz"
            img_full_path = os.path.join(tp_path, img_fn)
            
            if not os.path.exists(img_full_path):
                continue
                
            img_arr = sitk.GetArrayFromImage(sitk.ReadImage(img_full_path))
            print(f"Processing: {patient} | {tp} | {modality} ({num_lesions} lésions)")
            
            for l_id in range(1, num_lesions + 1):
                lesion_mask_3d = (mask_arr == l_id)
                areas = lesion_mask_3d.sum(axis=(1, 2))
                best_z = np.argmax(areas)
                
                # Extraction ROI
                slice_mask = lesion_mask_3d[best_z, :, :]
                ys, xs = np.where(slice_mask)
                if len(ys) < 2 or len(xs) < 2: continue
                
                roi = img_arr[best_z, ys.min():ys.max()+1, xs.min():xs.max()+1] * slice_mask[ys.min():ys.max()+1, xs.min():xs.max()+1]
                
                # Normalisation et Décomposition
                max_v = np.max(roi)
                roi_norm = roi / max_v if max_v > 0 else roi
                
                moments = decompose_2d(roi_norm, ORDRE_CIBLE) # Déjà arrondi à 3
                
                # Sauvegarde CSV
                csv_name = f"{patient}_{tp}_{modality}_L{l_id}_Cheby.csv"
                with open(os.path.join(current_out_dir, csv_name), "w", newline="") as f:
                    csv.writer(f).writerows(moments)

                # Validation rapide (Visualisation facultative pour 70 patients - décommenter si besoin)
                # reconstruction = reconstruct_2d(moments, roi_norm.shape)
                # mse = np.mean((roi_norm - reconstruction)**2)
                
print("\n[SUCCÈS] Toutes les lésions de tous les patients ont été extraites.")

Processing: P1 | T1 | T1 (18 lésions)
Processing: P1 | T1 | T2 (18 lésions)
Processing: P1 | T1 | FLAIR (18 lésions)
Processing: P1 | T2 | T1 (18 lésions)
Processing: P1 | T2 | T2 (18 lésions)
Processing: P1 | T2 | FLAIR (18 lésions)
Processing: P1 | T3 | T1 (14 lésions)
Processing: P1 | T3 | T2 (14 lésions)
Processing: P1 | T3 | FLAIR (14 lésions)
Processing: P10 | T1 | T1 (35 lésions)
Processing: P10 | T1 | T2 (35 lésions)
Processing: P10 | T1 | FLAIR (35 lésions)
Processing: P10 | T2 | T1 (32 lésions)
Processing: P10 | T2 | T2 (32 lésions)
Processing: P10 | T2 | FLAIR (32 lésions)
Processing: P11 | T1 | T1 (15 lésions)
Processing: P11 | T1 | T2 (15 lésions)
Processing: P11 | T1 | FLAIR (15 lésions)
Processing: P11 | T2 | T1 (10 lésions)
Processing: P11 | T2 | T2 (10 lésions)
Processing: P11 | T2 | FLAIR (10 lésions)
Processing: P12 | T1 | T1 (36 lésions)
Processing: P12 | T1 | T2 (36 lésions)
Processing: P12 | T1 | FLAIR (36 lésions)
Processing: P12 | T2 | T1 (42 lésions)
Processing